## Primer Design

One of the first things anyone learns in a molecular biology lab is how to design primers. The exact strategies vary a lot and are sometimes polymerase-specific. `coral` uses the Klavins' lab approach of targeting a specific melting temperature (Tm) and nothing else, with the exact Tm targeted being between 65°C and 72°C, the choice being personal preference. `coral` currently defaults to 72°C on the Phusion (modified Breslauer 1986) Tm calculator.

`coral.design_primer` is a function that takes in a `sequence.DNA` object and rapidly finds the 5' subsequence that is closest to the desired Tm (within a user-definable error range). If the entire sequence would make a primer with too low of a Tm, a descriptive error is produced.

For this tutorial, let's design primers that will amplify the gene EYFP.

In [1]:
import coral as cor

First we read in a plasmid from Havens et al. 2012 and isolate the EYFP sequence.

In [2]:
plasmid = cor.seqio.read_dna("../files_for_tutorial/maps/pGP4G-EYFP.ape")
eyfp_f = [f for f in plasmid.features if f.name == 'EYFP'][0]
eyfp = plasmid.extract(eyfp_f)
print len(eyfp)
eyfp

717


ATGGTGAGCAAGGGCGAGGAGCTGTTCACCGGGGTGGTGC ... CGCCGCCGGGATCACTCTCGGCATGGACGAGCTGTACAAG
TACCACTCGTTCCCGCTCCTCGACAAGTGGCCCCACCACG ... GCGGCGGCCCTAGTGAGAGCCGTACCTGCTCGACATGTTC

Designing primers is straightforward - you just call `design.design_primer` with a `sequence.DNA` object as the input.

In [3]:
# Forward and reverse, one at a time using design_primer()
forward = cor.design.primer(eyfp)
reverse = cor.design.primer(eyfp.reverse_complement())
# Both at once using design_primers()
forward, reverse = cor.design.primers(eyfp)
# design_primer has many options, including adding overhangs
custom_forward = cor.design.primer(eyfp, tm=65, min_len=12, 
                                   tm_undershoot=1, tm_overshoot=1, 
                                   end_gc=True, tm_parameters="santalucia98", 
                                   overhang=cor.DNA("GGGGGATCGAT"))
print forward
print
print custom_forward

ATGGTGAGCAAGGGCG

GGGGGATCGATATGGTGAGCAAGGGCGAGGAGCTGTTCAC


Designing primers and getting a string output is just the first step in primer design - we want to know whether the primers actually *work* and write them out to a file. The point of programming DNA is that you *never* copy and paste!

To simulate a PCR using the rules of molecular biology, use `coral.reaction.pcr`. The output is a subsequence of the template DNA - the features may not match the plasmid exactly (due to being truncated by the PCR), but the sequences match. If a primer would bind in multiple places (exact matches to the template), the pcr function will fail and give a useful message.

You can check for identical sequences using python's built in == operator.

In [4]:
amplicon = cor.reaction.pcr(plasmid, forward, reverse)
amplicon == eyfp

True

Now that we have verified that our primers should at least amplify the DNA that we want, let's write out our primers to file so they can be submitted to an oligo synthesis company.

In [5]:
# First we give our primers names (the `.name` attribute is empty by default)
forward.name = "EYFP_forward"
reverse.name = "EYFP_reverse"
# Then we write to file - a csv (comma separated value file)
cor.seqio.write_primers([forward, reverse], "./designed_primers.csv", ["Forward EYFP primer", "Reverse EYFP primer"])

The csv file can then be opened in a spreadsheet application like Excel or processed by a downstream program. This is the format of the csv:

In [6]:
import csv
with open("./designed_primers.csv", "r") as csv_file:
    reader = csv.reader(csv_file)
    lines = [line for line in reader]
for line in lines:
    print line

['name', 'sequence', 'notes']
['Forward EYFP primer', 'ATGGTGAGCAAGGGCG', '']
['Reverse EYFP primer', 'CTTGTACAGCTCGTCCATGCC', '']
